In [6]:
# imports
%run SMC_ABC_recoded.py
%matplotlib inline
import DataAnalysisFunctions as daf

%px import numpy as np
%px import scipy as sp
%px import random
%px from scipy import stats
%px from ipyparallel import CompositeError

rc[:]['create_parameter_vector'] = create_parameter_vector
rc[:]['singleTranscript'] = singleTranscript

# load experimental data
path_data = '/Users/baumgaer/ownCloud/GitHub/gene_transcription_SMC_ABC/experimental_data/'

data_0pME2 = np.loadtxt(path_data+'data_0pME2.csv')
data_5pME2 = np.loadtxt(path_data+'data_5pME2.csv')
data_7pME2 = np.loadtxt(path_data+'data_7pME2.csv')
data_10pME2 = np.loadtxt(path_data+'data_10pME2.csv')
data_14pME2 = np.loadtxt(path_data+'data_14pME2.csv')
data_20pME2 = np.loadtxt(path_data+'data_20pME2.csv')
data_100pME2 = np.loadtxt(path_data+'data_100pME2.csv')
data_1000pME2 = np.loadtxt(path_data+'data_1000pME2.csv')

mock_0pME2 = np.loadtxt(path_data+'mock_0pME2.csv')
mock_5pME2 = np.loadtxt(path_data+'mock_5pME2.csv')
mock_7pME2 = np.loadtxt(path_data+'mock_7pME2.csv')
mock_10pME2 = np.loadtxt(path_data+'mock_10pME2.csv')
mock_14pME2 = np.loadtxt(path_data+'mock_14pME2.csv')
mock_20pME2 = np.loadtxt(path_data+'mock_20pME2.csv')
mock_100pME2 = np.loadtxt(path_data+'mock_100pME2.csv')
mock_1000pME2 = np.loadtxt(path_data+'mock_1000pME2.csv')

# Using Sequential Monte Carlo Approximate Bayesian Computation to fit experimental data

##  Create a large start population by random sampling

Generate about 50000 candidate particles sampled from prior distributions. Simulate mRNA count only and save to file. Make the number of simulated cells larger than actual data sets. Before running SMC ABC take the start population, add noise and then find the best candidate particles for the SMC fitting. Reuse the start population for each fit be it real data or benchmark data. This should increase the sampling of the model and parameter space substantially.

The resulting arrays of the simulations can become very big, thus it is easier not to hold all 50000 particles into one file but save 2000 or so particles into one file. This large start population is created only once and the sampled candidates are resused later before every fit to creat a sufficient start population.

In [2]:
# parameters

# population size for SMC ABC
n_particles = 2000

# stop distance
stop = 0.2

# maximal number of iterations
iterations = 1

# path to save the results
path = '/Users/stephan/Desktop/gene_transcription_SMC_ABC/fits/'

# folder to temporally store individual particles to find a good start population
temp = '/Users/stephan/Desktop/temp/'

# path to candidate particles from the large start population
cand_path = '/Users/stephan/Desktop/gene_transcription_SMC_ABC/cand/'

In [10]:
# number of files to create
rounds = 2
start = 0
count = start
# sampled candidate particles per file
n_particles = 50
n_sim = 100
while count - start < rounds:
    print count - start
    res = smc_start_multi(n_particles,n_sim)
    np.save(cand_path+'res_'+str(count)+'.npy',res)
    count = count + 1

0
Duration: 3.13
1
Duration: 3.14


## Fit experimental data from estrogen dose response using the large sampled start population

Here exemplified on the 0 pM estrogen data set

### 0pM, no constraints on model topology

In [29]:
# compare data to blind shooting candidates
paths = [cand_path,temp]
dist_cand_0pM,par_cand_0pM = particle_distance_lut(Data0pM,Mock0pM,paths)

# save distances and parameters of all candidate particles
np.save(path + '0pM/dist_cand_0pM.npy',dist_cand_0pM)
np.save(path+'0pM/par_cand_0pM.npy',par_cand_0pM)

files = corrected_filelist(temp)
start_population = best_start_particles(files,dist_cand_0pM,n_particles)
np.save(path+'0pM/start_0pM_2000_corrected.npy',start_population)

File: 1 / 2
File: 2 / 2
Duration: 0.34 min
Duration: 0.01 min


In [3]:
start_population = np.load(path+'0pM/start_0pM_2000.npy')
start_population[0][:,0] = 1./start_population[0].shape[0]

In [4]:
alpha = 33.15
threshold = 0.6
save = [True,path+'0pM/','smc_0pM_']
smc = smc_abc(Data0pM,Mock0pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'0pM/smc_0pM_2000_lut.npy',smc)

# particles better than stop: 0
Min start distance: 0.811
Mean start distance: 3.313
Max start distance: 4.83
STD of start distance: 1.014
Mean of burst, tau and T: [ 64.449   9.297  63.678]
STD of burst, tau and T: [ 106.679    9.611   67.646]
Iteration: 1
Threshold distance 2.653


<string>:838: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
<string>:839: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Try: 0 First particles accepted: 71
Try: 1 More particles accepted: 85
Enough particles accepted: 85
# Created particles: 98
# particles better than stop: 0
Min distance of current population: 0.531
Mean distance of current population: 1.631
Max distance of current population: 2.57
Mean of burst, tau and T: [   5.598    2.194  173.988]
STD of burst, tau and T: [   6.104    2.147  190.286]
Duration of 1 th iteration 0.81 min
Last iteration finished, max distance: 2.56965204996
Duration of iterations: 0.81 min


### Fit 115 model to 0 pM 

The 115 model (two state with combined source of extrinsic noise of RNA pol II elongation rate and initiation rate) is by far the most frequent model for the low estrogen concentrations. Fitting this model with fixed topology allows reduce the dimensionality of the search space. The 115 model has four kinetic parameters. Fitting all data sets with the same model topology is a way to assess the dependency of the model parameters on experimental conditions.

Any other model from the list of allowed models can be fitted in a similar way by selecting the model topology of interest.

In [12]:
# find the best 1000 particles in the posterior distribution for the 0 pM data set
ind = np.where(smc_0pM[0][:,1,-1] == 5)[0]
ind = ind[0:1000]
start_population = [smc_0pM[0][ind,:,-1],smc_0pM[1][:,:,ind],smc_0pM[2][ind,:,-1],smc_0pM[3][:,:,ind]]
start_population[0][:,0] = 1./1000.

In [13]:
alpha = 33.15

# the threshold value defines the likelihood for changes on the model space. A value of one prohibits any jumps at 
# all and thus keeps the model topology fixed.
threshold = 1.
save = [True,path+'0pM/','smc_0pM_115_']
smc = smc_abc(Data0pM,Mock0pM,start_population,iterations,stop,threshold=threshold,alpha = alpha,save = save)
np.save(path+'0pM/smc_0pM_1000_115.npy',smc)

# particles better than stop: 0
Min start distance: 0.504
Mean start distance: 0.61
Max start distance: 0.64
STD of start distance: 0.024
Mean of burst, tau and T: [   4.168    1.618  309.055]
STD of burst, tau and T: [  2.836   1.104  83.979]
Iteration: 1
Threshold distance 0.591
Try: 0 First particles accepted: 15
Try: 1 More particles accepted: 20
Try: 2 More particles accepted: 32
Try: 3 More particles accepted: 37
Try: 4 More particles accepted: 52
Try: 5 More particles accepted: 67
Try: 6 More particles accepted: 85
Try: 7 More particles accepted: 100
Try: 8 More particles accepted: 116
Try: 9 More particles accepted: 135
Try: 10 More particles accepted: 143
Slow creation of new particles, reduced number of tries to: 15
Try: 11 More particles accepted: 155
Try: 12 More particles accepted: 171
Try: 13 More particles accepted: 180
Try: 14 More particles accepted: 192
Not enough particles accepted: 192
# Created particles: 10692
# particles better than stop: 0
Min distance of curren